## Imports

In [1]:
import geopandas as gpd
import os, sys, copy, shutil, subprocess
import pathlib as pl
import fiona
import descartes
import json
import numpy as np
import sys
import logging
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append('..')
from src import config
from src.data import process_inputs
from src.data import utils 
from src.grass_functions import *

### Get HUC number

In [2]:
huc_list_path = pl.Path(os.getcwd()).parent/'huc12.txt'
with open(huc_list_path, 'r') as txt_file:
    huc_list = []
    txt_reader = txt_file.read().splitlines() 
    for huc in txt_reader:
        huc_list.append(huc)

In [3]:
huc = huc_list[0]
logger.info("Selected 1st huc from list " + huc)

[2022-03-21 18:09:33,227] [INFO] [601608545] : Selected 1st huc from list 110300060208


### Configure

In [4]:
os.environ["APP_ENV"] = 'LOCAL'
config = config.create_config(huc)

### Download Data (NHD, WBD, DEM)

In [5]:
config = process_inputs.main(huc=huc, config=config, overwrite=True)

[2022-03-21 18:09:39,054] [INFO] [process_inputs] : downloading NHD Plus GDB for HUC 1103
[2022-03-21 18:09:39,887] [INFO] [process_inputs] : opening: https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHDPlusHR/Beta/GDB/NHDPLUS_H_1103_HU4_GDB.zip
[2022-03-21 18:09:54,710] [INFO] [process_inputs] : saving to: /home/data/nhd
[2022-03-21 18:10:15,623] [INFO] [process_inputs] : reprojecting HUC 12 boundary shapefile to EPSG 5070
[2022-03-21 18:10:18,463] [INFO] [process_inputs] : extracting NHDFlowline layer and reprojecting
[2022-03-21 18:12:26,017] [WARNING] [geodataframe] : Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[2022-03-21 18:12:26,058] [INFO] [process_inputs] : downloading relevant DEMs
[2022-03-21 18:14:06,609] [INFO] [process_inputs] : reprojecting and merging DEMs
[2022-03-21 18:14:06,919] [INFO] [process_inputs] : checking DEM coverage for /home/data/dem/res10m/huc110300060208_dem_sr5070.vrt
[2022-03-21 18:15:22,875] [WARNING] [geodat

### Start new grass session

In [6]:
config = create_grass_session(huc,config, overwrite=True)

[2022-03-21 18:15:22,944] [INFO] [utils] : creating timestamped log directory: /home/logs/huc110300060208/20220321-181522
[2022-03-21 18:15:22,971] [INFO] [utils] : setting up GRASS session for HUC 110300060208, EPSG 5070
[2022-03-21 18:15:25,499] [INFO] [utils] : GRASS session set up: {'GISDBASE': '/home/grassdata', 'LOCATION_NAME': 'huc110300060208_sr5070', 'MAPSET': 'ML'}


### Import DEM, convert vertical units, mask and export Bare Earth DEM

In [7]:
config = dem_to_grass(config, overwrite = True)

Importing raster map <huc110300060208_dem_sr5070>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
[2022-03-21 18:17:20,090] [INFO] [grass_functions] : Checking vertical units and converting to :feet
[2022-03-21 18:17:20,237] [INFO] [grass_functions] : converting vertical units to feet. Raw DEM * 3.280839895013123
Rename raster <new_rast> to <huc110300060208_dem_sr5070>
W-E size of neighborhood is 7 cells.
S-N size of neighborhood is 7 cells.
Input data range is 1577.971351 to 2631.876988.
Input data type is 'double' (8 bytes) and output data type is 'double' (8
bytes).
Minimal estimated memory usage is 1.234 MB.
Interpolating:
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.fill.stats complete. Processing time was 0h2m38s.
Rename raster <dem_tmp> to <huc110300060208_dem_sr5070>
[2022-03-21 18:22:16,338

### Create Curvature Lines along streams in Grass GIS (for hydro-enforcing)

In [9]:
config = create_draft_stream_accum(config,overwrite = True)

[2022-03-21 17:46:51,178] [INFO] [grass_functions] : performing initial pass of watershed analysis
SECTION 1a (of 5): Initiating Memory.
SECTION 1b (of 5): Determining Offmap Flow.
   0   4   8  12  16  20  24  28  32  36  40  44  48  52  56  60  64  68  72  76  80  84  88  92  96 100
SECTION 2: A* Search.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100
SECTION 3: Accumulating Surface Flow with SFD.
   1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71  73  75  77  79  81  83  85  87  89  91  93  95  97  99 100
SECTION 4: Watershed determination.
   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  9

In [10]:
adjust_nhd_lines(config,overwrite = True)

[2022-03-21 17:48:10,582] [INFO] [grass_functions] : combining watershed lines and nhd lines
Building topology for vector map <nhd_flowline_poly@ML>...
Registering primitives...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Building topology for vector map <nhd_flowline_pts@ML>...
Registering primitives...
v.to.points complete. 36394 points written to output vector map.
[2022-03-21 17:49:24,811] [INFO] [grass_functions] : capturing elevations along nhd lines
Processing features...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98WARNING: Undefined height for feature 36127. Skipping.
 100
Copying attribute tables...
Building topology for vector map <nhd_flowline_pts_3d@ML>...
Registering primitives...
v.drape complete. T: 2364.722625 B: 2124.903121.


In [ ]:
create_hydro_connectors(config,overwrite = True)

[2022-03-21 17:51:20,404] [INFO] [grass_functions] : exporting nhd lines for road and dam analysis
Fetching data...
Fetching data...
Fetching data...


228 Rd


Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...


L Rd


Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...


...

Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...


K Rd


Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...
Fetching data...


228 Rd


Fetching data...
Fetching data...
Fetching data...
Fetching data...


In [ ]:
config = topo_carve(config,overwrite = True)

### Create Accumulation, Flow Direction and Flow lines

In [ ]:
config = develop_watershed_data(config, overwrite = True)

## End

### Create 1sq mile threshold

In [27]:
def delineate_at_threshold(config,overwrite = False):
    accum = config.accum
    threshold = config.thresholds['stream']
    reg = gs.parse_command('g.region', raster=accum, flags='pgm',align=accum)
    cell_area = (float(reg.nsres)*float(reg.ewres))
    target_accum_cells = threshold / cell_area
    gs.run_command('r.mapcalc',overwrite=True,\
                      expression='above_thresh = if({0} < {1},null(),{0})'.format(accum, target_accum_cells))
    gs.run_command('r.mapcalc',overwrite=True,\
                      expression='outlet = if({0} = nmin({0}),1,null())'.format(accum))
    gs.run_command('r.to.vect',overwrite=True,\
                          input = 'outlet', output = 'outlet_vec',\
                          type='point')
    v_basin = delineate_at_point('outlet',config.drain,'thresholds')
    gs.run_command('v.out.ogr', input=  v_basin ,type = 'area',output = out_dir/'{}.gpkg'.format(v_basin), format = 'GPKG')
    
def delineate_at_point(point_v,flow_dir,basin_name):
    gs.run_command('r.stream.basins',overwrite=True,\
              direction=flow_dir,points=point_v,\
              basins = 'r_{}'.format(basin_name))
    gs.run_command('r.to.vect',input = 'r_{}'.format(basin_name),output = 'v_{}'.format(basin_name), type = 'area')
    gs.run_command('v.clean',input = 'v_{}'.format(basin_name),threshold = 5000,tool='rmarea',output = '{}_clean'.format(v_basins))
    return 'v_{}'.format(basin_name)
    

In [28]:
delineate_at_threshold(config,overwrite = True)

Extracting points...
   0   3   6   9  12  15  18  21  24  27

KeyboardInterrupt: 